In [8]:
# Here we import all libraries
import numpy as np
import gym
import matplotlib.pyplot as plt
import os
import torch
import random
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from collections import deque 
import torchvision as tv
import torch.nn.functional as F
import sys
env = gym.make("ALE/Pong-v5")

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [10]:
#Hyperparameters
episodes = 20000*10
eps = 1.0
learning_rate = 0.001
tot_rewards = []
tot_loss = []
decay_val = 0.0001
mem_size = 100000
batch_size = 128
gamma = 0.99
update_target = 100
max_steps = 200
PATH = "./saved_models/pong"

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(NeuralNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.grayscale = tv.transforms.Grayscale()
        self.conv1 = nn.Conv2d(1,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(29008, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, action_size)
        
    def forward(self, x):
        x = self.grayscale(x)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [12]:
print(env.observation_space.shape[2])
model = NeuralNetwork(env.observation_space.shape[2], env.action_space.n).to(device)
target = NeuralNetwork(env.observation_space.shape[2], env.action_space.n).to(device)

opt = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
replay_buffer = deque(maxlen=mem_size)

3


In [13]:
def compute_td_loss(batch_size):
    state, next_state, reward, done, action = zip(*random.sample(replay_buffer, batch_size))
    state = torch.stack(list(state), dim=0).squeeze(1)
    state= state.reshape(batch_size, 3, 210, 160).to(device)
    next_state = torch.from_numpy(np.array(next_state)).reshape(batch_size, 3, 210, 160).type(torch.float32).to(device)
    reward = torch.from_numpy(np.array(reward)).to(device)
    done = torch.from_numpy(np.array(done)).long().to(device)
    action = torch.from_numpy(np.array(action)).type(torch.int64).to(device)
    q_values = model(state)
    next_q_values = target(next_state)
    q_vals = q_values.gather(dim=-1, index=action.reshape(-1,1))
    max_next_q_values = torch.max(next_q_values,-1)[0].detach()
    loss = ((reward + gamma*max_next_q_values*(1-done) - q_vals.squeeze())**2).mean()
    opt.zero_grad()
    loss.backward()
    opt.step()
    return loss
    

In [ ]:
if os.path.exists(PATH):
    model.load_state_dict(torch.load(PATH))
else:
    frame_index = 0
    for i in range(episodes):
        state = torch.tensor(env.reset(), dtype=torch.float32).unsqueeze(0)
        state= state.reshape(1, 3, 210, 160)
        done = False
        steps = 0
        eps_rew = 0 
        eps_loss = 0
        while not done and steps<max_steps:
            print("frame_index = ", frame_index, "episode = ", i)
            if np.random.uniform(0,1)<eps:
                action = env.action_space.sample()
            else:
                action = torch.argmax(model(state.to(device))).cpu().detach().numpy()


            next_state, reward, done, info = env.step(action)
            replay_buffer.append((state, next_state, reward, done, action))
#             if len(replay_buffer)>batch_size and steps%4==0:
            if len(replay_buffer)==mem_size:
                loss = compute_td_loss(batch_size)
                eps_loss += loss.cpu().detach().numpy()
            eps = eps/(1 + decay_val)
            eps_rew += reward 

            if steps%50==0:
                target.load_state_dict(model.state_dict())

            if done:
                tot_rewards.append(eps_rew)
                break

            state = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0)
            state= state.reshape(1, 3, 210, 160)
            steps += 1
            frame_index += 1
        tot_rewards.append(eps_rew)
        tot_loss.append(eps_loss)

        if(frame_index%5000)==0:
            np.savetxt("tot_rewards.csv", np.array(tot_rewards), delimiter=' ', fmt='%s')
            torch.save(model.state_dict(), PATH)
    torch.save(model.state_dict(), PATH)

frame_index =  0 episode =  0
frame_index =  1 episode =  0
frame_index =  2 episode =  0
frame_index =  3 episode =  0
frame_index =  4 episode =  0
frame_index =  5 episode =  0
frame_index =  6 episode =  0
frame_index =  7 episode =  0
frame_index =  8 episode =  0
frame_index =  9 episode =  0
frame_index =  10 episode =  0
frame_index =  11 episode =  0
frame_index =  12 episode =  0
frame_index =  13 episode =  0
frame_index =  14 episode =  0
frame_index =  15 episode =  0
frame_index =  16 episode =  0
frame_index =  17 episode =  0
frame_index =  18 episode =  0
frame_index =  19 episode =  0
frame_index =  20 episode =  0
frame_index =  21 episode =  0
frame_index =  22 episode =  0
frame_index =  23 episode =  0
frame_index =  24 episode =  0
frame_index =  25 episode =  0
frame_index =  26 episode =  0
frame_index =  27 episode =  0
frame_index =  28 episode =  0
frame_index =  29 episode =  0
frame_index =  30 episode =  0
frame_index =  31 episode =  0
frame_index =  32 

C:\Users\thoma\anaconda3\envs\torch_1\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


frame_index =  152 episode =  0
frame_index =  153 episode =  0
frame_index =  154 episode =  0
frame_index =  155 episode =  0
frame_index =  156 episode =  0
frame_index =  157 episode =  0
frame_index =  158 episode =  0
frame_index =  159 episode =  0
frame_index =  160 episode =  0
frame_index =  161 episode =  0
frame_index =  162 episode =  0
frame_index =  163 episode =  0
frame_index =  164 episode =  0
frame_index =  165 episode =  0
frame_index =  166 episode =  0
frame_index =  167 episode =  0
frame_index =  168 episode =  0
frame_index =  169 episode =  0
frame_index =  170 episode =  0
frame_index =  171 episode =  0
frame_index =  172 episode =  0
frame_index =  173 episode =  0
frame_index =  174 episode =  0
frame_index =  175 episode =  0
frame_index =  176 episode =  0
frame_index =  177 episode =  0
frame_index =  178 episode =  0
frame_index =  179 episode =  0
frame_index =  180 episode =  0
frame_index =  181 episode =  0
frame_index =  182 episode =  0
frame_in

frame_index =  599 episode =  2
frame_index =  600 episode =  3
frame_index =  601 episode =  3
frame_index =  602 episode =  3
frame_index =  603 episode =  3
frame_index =  604 episode =  3
frame_index =  605 episode =  3
frame_index =  606 episode =  3
frame_index =  607 episode =  3
frame_index =  608 episode =  3
frame_index =  609 episode =  3
frame_index =  610 episode =  3
frame_index =  611 episode =  3
frame_index =  612 episode =  3
frame_index =  613 episode =  3
frame_index =  614 episode =  3
frame_index =  615 episode =  3
frame_index =  616 episode =  3
frame_index =  617 episode =  3
frame_index =  618 episode =  3
frame_index =  619 episode =  3
frame_index =  620 episode =  3
frame_index =  621 episode =  3
frame_index =  622 episode =  3
frame_index =  623 episode =  3
frame_index =  624 episode =  3
frame_index =  625 episode =  3
frame_index =  626 episode =  3
frame_index =  627 episode =  3
frame_index =  628 episode =  3
frame_index =  629 episode =  3
frame_in

frame_index =  888 episode =  4
frame_index =  889 episode =  4
frame_index =  890 episode =  4
frame_index =  891 episode =  4
frame_index =  892 episode =  4
frame_index =  893 episode =  4
frame_index =  894 episode =  4
frame_index =  895 episode =  4
frame_index =  896 episode =  4
frame_index =  897 episode =  4
frame_index =  898 episode =  4
frame_index =  899 episode =  4
frame_index =  900 episode =  4
frame_index =  901 episode =  4
frame_index =  902 episode =  4
frame_index =  903 episode =  4
frame_index =  904 episode =  4
frame_index =  905 episode =  4
frame_index =  906 episode =  4
frame_index =  907 episode =  4
frame_index =  908 episode =  4
frame_index =  909 episode =  4
frame_index =  910 episode =  4
frame_index =  911 episode =  4
frame_index =  912 episode =  4
frame_index =  913 episode =  4
frame_index =  914 episode =  4
frame_index =  915 episode =  4
frame_index =  916 episode =  4
frame_index =  917 episode =  4
frame_index =  918 episode =  4
frame_in

...

frame_index =  10732 episode =  53
frame_index =  10733 episode =  53
frame_index =  10734 episode =  53
frame_index =  10735 episode =  53
frame_index =  10736 episode =  53
frame_index =  10737 episode =  53
frame_index =  10738 episode =  53
frame_index =  10739 episode =  53
frame_index =  10740 episode =  53
frame_index =  10741 episode =  53
frame_index =  10742 episode =  53
frame_index =  10743 episode =  53
frame_index =  10744 episode =  53
frame_index =  10745 episode =  53
frame_index =  10746 episode =  53
frame_index =  10747 episode =  53
frame_index =  10748 episode =  53
frame_index =  10749 episode =  53
frame_index =  10750 episode =  53
frame_index =  10751 episode =  53
frame_index =  10752 episode =  53
frame_index =  10753 episode =  53
frame_index =  10754 episode =  53
frame_index =  10755 episode =  53
frame_index =  10756 episode =  53
frame_index =  10757 episode =  53
frame_index =  10758 episode =  53
frame_index =  10759 episode =  53
frame_index =  10760

frame_index =  11033 episode =  55
frame_index =  11034 episode =  55
frame_index =  11035 episode =  55
frame_index =  11036 episode =  55
frame_index =  11037 episode =  55
frame_index =  11038 episode =  55
frame_index =  11039 episode =  55
frame_index =  11040 episode =  55
frame_index =  11041 episode =  55
frame_index =  11042 episode =  55
frame_index =  11043 episode =  55
frame_index =  11044 episode =  55
frame_index =  11045 episode =  55
frame_index =  11046 episode =  55
frame_index =  11047 episode =  55
frame_index =  11048 episode =  55
frame_index =  11049 episode =  55
frame_index =  11050 episode =  55
frame_index =  11051 episode =  55
frame_index =  11052 episode =  55
frame_index =  11053 episode =  55
frame_index =  11054 episode =  55
frame_index =  11055 episode =  55
frame_index =  11056 episode =  55
frame_index =  11057 episode =  55
frame_index =  11058 episode =  55
frame_index =  11059 episode =  55
frame_index =  11060 episode =  55
frame_index =  11061

frame_index =  11339 episode =  56
frame_index =  11340 episode =  56
frame_index =  11341 episode =  56
frame_index =  11342 episode =  56
frame_index =  11343 episode =  56
frame_index =  11344 episode =  56
frame_index =  11345 episode =  56
frame_index =  11346 episode =  56
frame_index =  11347 episode =  56
frame_index =  11348 episode =  56
frame_index =  11349 episode =  56
frame_index =  11350 episode =  56
frame_index =  11351 episode =  56
frame_index =  11352 episode =  56
frame_index =  11353 episode =  56
frame_index =  11354 episode =  56
frame_index =  11355 episode =  56
frame_index =  11356 episode =  56
frame_index =  11357 episode =  56
frame_index =  11358 episode =  56
frame_index =  11359 episode =  56
frame_index =  11360 episode =  56
frame_index =  11361 episode =  56
frame_index =  11362 episode =  56
frame_index =  11363 episode =  56
frame_index =  11364 episode =  56
frame_index =  11365 episode =  56
frame_index =  11366 episode =  56
frame_index =  11367

frame_index =  11588 episode =  57
frame_index =  11589 episode =  57
frame_index =  11590 episode =  57
frame_index =  11591 episode =  57
frame_index =  11592 episode =  57
frame_index =  11593 episode =  57
frame_index =  11594 episode =  57
frame_index =  11595 episode =  57
frame_index =  11596 episode =  57
frame_index =  11597 episode =  57
frame_index =  11598 episode =  57
frame_index =  11599 episode =  57
frame_index =  11600 episode =  58
frame_index =  11601 episode =  58
frame_index =  11602 episode =  58
frame_index =  11603 episode =  58
frame_index =  11604 episode =  58
frame_index =  11605 episode =  58
frame_index =  11606 episode =  58
frame_index =  11607 episode =  58
frame_index =  11608 episode =  58
frame_index =  11609 episode =  58
frame_index =  11610 episode =  58
frame_index =  11611 episode =  58
frame_index =  11612 episode =  58
frame_index =  11613 episode =  58
frame_index =  11614 episode =  58
frame_index =  11615 episode =  58
frame_index =  11616

frame_index =  11896 episode =  59
frame_index =  11897 episode =  59
frame_index =  11898 episode =  59
frame_index =  11899 episode =  59
frame_index =  11900 episode =  59
frame_index =  11901 episode =  59
frame_index =  11902 episode =  59
frame_index =  11903 episode =  59
frame_index =  11904 episode =  59
frame_index =  11905 episode =  59
frame_index =  11906 episode =  59
frame_index =  11907 episode =  59
frame_index =  11908 episode =  59
frame_index =  11909 episode =  59
frame_index =  11910 episode =  59
frame_index =  11911 episode =  59
frame_index =  11912 episode =  59
frame_index =  11913 episode =  59
frame_index =  11914 episode =  59
frame_index =  11915 episode =  59
frame_index =  11916 episode =  59
frame_index =  11917 episode =  59
frame_index =  11918 episode =  59
frame_index =  11919 episode =  59
frame_index =  11920 episode =  59
frame_index =  11921 episode =  59
frame_index =  11922 episode =  59
frame_index =  11923 episode =  59
frame_index =  11924